<a href="https://colab.research.google.com/github/sanjaybora04/Nike_Masked_LM/blob/main/nike_masked_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Required packages
- [PyTorch](https://pytorch.org/get-started/locally/): Deep learning framework.
- Hugging Face Transformers(https://huggingface.co/docs/transformers/installation): To use pretrained state-of-the-art models.
- [Hugging Face Datasets](https://pypi.org/project/datasets/): Use public Hugging Face datasets
- [IPywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html): For interactive notebook widgets

In [1]:
!git clone https://github.com/uptrain-ai/uptrain
!mv uptrain/examples/llm_bert/* /content/
!rm -rf uptrain

Cloning into 'uptrain'...
remote: Enumerating objects: 1796, done.
remote: Counting objects: 100% (927/927), done.
remote: Compressing objects: 100% (530/530), done.
remote: Total 1796 (delta 598), reused 621 (delta 393), pack-reused 869
Receiving objects: 100% (1796/1796), 6.66 MiB | 11.40 MiB/s, done.
Resolving deltas: 100% (1038/1038), done.


In [2]:
!pip install torch transformers[torch] datasets ipywidgets nltk uptrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3

In [3]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
import uptrain

from model_constants import *
from model_train import retrain_model
from helper_funcs import *

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Define few cases to test our model performance before and after retraining.

In [4]:
testing_texts = [
    "Nike shoes are very [MASK].",
    "Nike shoes are good for [MASK].",
    "Nike shoes make me look [MASK].",
    "Nike products are [MASK].",
    "Nike's shoes [MASK] last long.",
    "Nike shoes look [MASK].",
    "Nike is a [MASK] brand."
]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
original_model_outputs = [test_model(model, x) for x in testing_texts]

Let's use Nike onlinestore customer reviews from Kaggle and filter data using UpTrain signals to retrain our model upon. Please download the data from the [link](https://www.kaggle.com/datasets/tinkuzp23/nike-onlinestore-customer-reviews?resource=download) and unzip it here.
  

In [6]:
!wget https://raw.githubusercontent.com/sanjaybora04/Nike_Masked_LM/main/web_scraped.csv

--2023-02-20 18:59:35--  https://raw.githubusercontent.com/sanjaybora04/Nike_Masked_LM/main/web_scraped.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65658 (64K) [text/plain]
Saving to: ‘web_scraped.csv’

web_scraped.csv     100%[===================>]  64.12K  --.-KB/s    in 0.01s   

2023-02-20 18:59:35 (5.61 MB/s) - ‘web_scraped.csv’ saved [65658/65658]



In [7]:
# Create Nike review training dataset
nike_attrs = {
    "version": "0.1.0",
    'source': "nike review dataset",
    'url': 'https://www.kaggle.com/datasets/tinkuzp23/nike-onlinestore-customer-reviews?resource=download',
}
# Download the dataset from the url, zip it and copy the csv file here
nike_reviews_dataset = create_dataset_from_csv("web_scraped.csv", "Content", "nike_reviews_data.json")

In [8]:
def nike_positive_sentiment_func(inputs, outputs, gts=None, extra_args={}):
    is_positives = []
    for input in inputs["text"]:
        txt = input.lower()
        sia = SentimentIntensityAnalyzer()
        score = sia.polarity_scores(txt)

        is_negative = score['pos'] < 0.25
        for neg_adj in ['expensive', 'worn', 'cheap', 'inexpensive', 'dirty', 'bad', 'filthy', 'worse']:
            if neg_adj in txt:
                is_negative = True

        is_positives.append(bool(1-is_negative))
    return is_positives

cfg = {
    'checks': [{
        'type': uptrain.Anomaly.EDGE_CASE,
        "signal_formulae": uptrain.Signal("Nike Positive Sentiment", nike_positive_sentiment_func)
    }],

    # Define where to save the retraining dataset
    'retraining_folder': "uptrain_smart_data",
    
    # Define when to retrain, define a large number because we are using UpTrain just to create retraining dataset
    'retrain_after': 10000000000
}

framework = uptrain.Framework(cfg)

In [9]:
with open(nike_reviews_dataset) as f:
    all_data = json.load(f)

for sample in all_data['data']:
    inputs = {'data': {'text': [sample['text']]}}
    framework.log(inputs = inputs, outputs = None)

50  edge cases identified out of  78  total samples
100  edge cases identified out of  183  total samples


In [10]:
print("Number of samples filtered for retraining: ", len(pd.read_csv("uptrain_smart_data/1/smart_data.csv")))
retraining_dataset = create_dataset_from_csv("uptrain_smart_data/1/smart_data.csv", "text", "retrain_dataset.json", min_samples=1000)

Number of samples filtered for retraining:  148


In [11]:
retrain_model(model, retraining_dataset)
retrained_model_outputs = [test_model(model, x) for x in testing_texts]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5a6ec89ec820522d/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

...

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 148
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 9
  Number of trainable parameters = 66985530


>>>Before training, Perplexity: 60.05


Epoch,Training Loss,Validation Loss
1,No log,3.908923
2,No log,3.392929
3,No log,3.503094


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17
  Batch size = 64


Training completed. Do not forget to shar

>>>After training, Perplexity: 26.17


In [13]:
[testing_texts,original_model_outputs, retrained_model_outputs]

[['Nike shoes are very [MASK].',
  'Nike shoes are good for [MASK].',
  'Nike shoes make me look [MASK].',
  'Nike products are [MASK].',
  "Nike's shoes [MASK] last long.",
  'Nike shoes look [MASK].',
  'Nike is a [MASK] brand.'],
 [['popular', 'expensive', 'durable', 'common', 'comfortable'],
  ['walking', 'cycling', 'wear', 'shoes', 'surfing'],
  ['sexy', 'ridiculous', 'better', 'uncomfortable', 'nice'],
  ['discontinued', 'available', 'manufactured', 'sold', 'unavailable'],
  ['would', 'will', 'should', 'could', 'must'],
  ['nice', 'sexy', 'ridiculous', 'cute', 'expensive'],
  ['nike', 'clothing', 'global', 'british', 'shoe']],
 [['popular', 'expensive', 'durable', 'comfortable', 'good'],
  ['walking', 'nike', 'cycling', 'riding', 'wear'],
  ['sexy', 'better', 'good', 'nice', 'uncomfortable'],
  ['available', 'discontinued', 'sold', 'manufactured', 'distributed'],
  ['would', 'will', 'could', 'should', 'might'],
  ['nice', 'sexy', 'good', 'amazing', 'cute'],
  ['nike', 'global', '